<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4177f285a3a7421aa929d19f3a7a5997822535bf53af5810c4ffcbb745e01021
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-01 15:23:59
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 48.20 K (0.34%)
Current PnL: -25.53 L (-16.28%)
CY Booked + Current PnL: -11.24 L (-7.17%)
-------------------
Total profit:  1.60 L
Total loss:  -27.13 L
-------------------
Total Booked + Current PnL: 15.74 L (12.13%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.95%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 90.50 L (63.04%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.44,14.05,6.44,21.40,19496.0,37300.0,302726.0,5.82,68.0,X-LC,5.15,12.0,1.91,2.11,21.27,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.30,-20.04,25.18,0.09,20290.0,-20199.0,80578.0,86.98,37.0,M-SC,3.37,253.0,-1.00,0.56,5.86,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,1.63,8.22,10.60,19.69,21996.0,15768.0,207510.0,-2.32,53.0,X-MC,2.07,78.0,0.72,1.45,15.58,XY25,NTT,AC
49,MASFIN,398.61,-1.27,-2.17,24.76,22.05,23732.0,-2130.0,95850.0,-15.54,53.0,H-SC,6.25,164.0,-0.09,0.67,39.32,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,3.57,-15.85,18.86,0.02,27703.0,-27669.0,146890.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.77,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,14.42,-27.27,55.24,12.90,87783.0,-59584.0,158912.0,134.38,74.0,H-SC,11.73,161.0,-0.68,1.11,32.13,OX40N,NTT,FINANCE
6,ATULAUTO,844.00,6.40,-19.29,80.36,45.57,127855.0,-38029.0,159103.0,3440.54,63.0,M-SC,6.51,244.0,-0.30,1.11,12.80,XY24,NTT,AUTO
25,FINCABLES,1641.55,4.92,-0.38,108.70,107.91,159881.0,-559.0,147085.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.02,9.67,OX40N,ATH,CABLES
70,SYMPHONY,1306.00,4.04,-29.77,42.34,-0.03,50889.0,-50949.0,120192.0,-43.61,68.0,M-SC,15.77,206.0,-1.00,0.84,9.23,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,3.57,-15.85,18.86,0.02,27703.0,-27669.0,146890.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.77,OX40N,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-9.50,-10.62,23.94,10.78,51425.0,-25510.0,214808.0,-46.87,18.0,X-LC,2.46,5.0,-0.50,1.50,0.00,X40,NTT,FMCG
78,UNITDSPR,1644.00,-2.75,3.14,17.09,20.77,41270.0,7344.0,241488.0,-5.80,44.0,X-MC,1.45,62.0,0.18,1.68,10.27,X40N,NTT,BREWERIES
26,GILLETTE,11206.78,-2.42,-3.58,37.34,32.42,91408.0,-9096.0,244800.0,-20.76,45.0,X-SC,11.60,85.0,-0.10,1.71,10.98,X40,ATH,FMCG
55,RAJESHEXPO,518.00,-2.29,-65.34,188.69,0.07,89720.0,-89628.0,47549.0,1642.42,38.0,L-SC,4.51,268.0,-1.00,0.33,18.26,OX40N,NTT,JEWELLERY
2,ABBOTINDIA,35195.00,-1.81,-4.39,23.49,18.07,40168.0,-7855.0,171000.0,-18.20,47.0,X-MC,7.66,63.0,-0.20,1.19,13.31,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,4.92,-0.38,108.7,107.91,159881.0,-559.0,147085.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.02,9.67,OX40N,ATH,CABLES
57,RECLTD,446.00,3.14,-0.38,21.2,20.74,42909.0,-770.0,202400.0,31.15,66.0,M-MC,8.24,189.0,-0.02,1.41,10.20,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,4.92,-0.38,108.70,107.91,159881.0,-559.0,147085.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.02,9.67,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,3.57,-15.85,18.86,0.02,27703.0,-27669.0,146890.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.77,OX40N,NTT,PAINTS
47,KANSAINER,340.00,1.93,-19.90,41.67,13.47,90007.0,-53667.0,216000.0,-67.15,60.0,H-SC,6.29,158.0,-0.60,1.50,9.83,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.30,-20.04,25.18,0.09,20290.0,-20199.0,80578.0,86.98,37.0,M-SC,3.37,253.0,-1.00,0.56,5.86,OX40N,NTT,DURABLES
66,SIS,528.00,-0.52,-23.83,59.20,21.26,50066.0,-26461.0,84571.0,1995.85,50.0,H-SC,4.72,166.0,-0.53,0.59,14.26,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.47,9.57,57.22,72.27,106240.0,16221.0,185669.0,-6.08,62.0,M-LC,5.43,99.0,0.15,1.29,16.34,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.47,9.57,57.22,72.27,106240.0,16221.0,185669.0,-6.08,62.0,M-LC,5.43,99.0,0.15,1.29,16.34,XR,NTT,IT
25,FINCABLES,1641.55,4.92,-0.38,108.70,107.91,159881.0,-559.0,147085.0,-18.93,58.0,M-SC,11.32,216.0,-0.00,1.02,9.67,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.14,-5.54,118.85,106.72,138459.0,-6837.0,116499.0,-53.30,39.0,H-SC,6.13,139.0,-0.05,0.81,16.54,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.07,-3.51,40.57,35.63,82039.0,-7359.0,202215.0,-17.18,44.0,H-MC,2.82,119.0,-0.09,1.41,12.58,AR,ATH,PHARMA
49,MASFIN,398.61,-1.27,-2.17,24.76,22.05,23732.0,-2130.0,95850.0,-15.54,53.0,H-SC,6.25,164.0,-0.09,0.67,39.32,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-9.50,-10.62,23.94,10.78,51425.0,-25510.0,214808.0,-46.87,18.0,X-LC,2.46,5.0,-0.50,1.50,0.00,X40,NTT,FMCG
22,DIXON,18931.72,0.62,-15.57,55.47,31.26,108073.0,-35936.0,194832.0,-60.59,33.0,X-MC,16.54,56.0,-0.33,1.36,3.48,X40N,ATH,IT
34,HONAUT,58357.33,0.90,-18.71,76.31,43.32,101034.0,-30472.0,132400.0,-30.16,34.0,X-SC,7.49,90.0,-0.30,0.92,1.66,X40N,ATH,ELECTRICAL
52,PAGEIND,51605.08,-1.10,-9.77,44.75,30.62,95720.0,-23154.0,213900.0,-35.86,34.0,X-MC,15.33,55.0,-0.24,1.49,0.24,X40,ATH,APPARELS
7,AWL,485.00,0.72,-22.54,102.93,57.19,260025.0,-73504.0,252623.0,-63.37,37.0,X-MC,7.14,58.0,-0.28,1.76,1.38,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.92,-16.94,51.11,25.51,79003.0,-31520.0,154575.0,-5.19,44.0,H-LC,0.91,49.0,-0.40,1.08,13.91,AR,ATH,ELECTRICAL
78,UNITDSPR,1644.0,-2.75,3.14,17.09,20.77,41270.0,7344.0,241488.0,-5.80,44.0,X-MC,1.45,62.0,0.18,1.68,10.27,X40N,NTT,BREWERIES
12,BERGEPAINT,680.0,-0.73,-5.11,27.66,21.14,59669.0,-11616.0,215723.0,-11.79,42.0,X-MC,1.82,74.0,-0.19,1.50,19.80,XY24,NTT,PAINTS
20,DABUR,735.0,-0.74,-2.27,47.04,43.70,114038.0,-5641.0,242427.0,-10.34,50.0,X-MC,1.97,72.0,-0.05,1.69,12.43,XY24,BTT,FMCG
82,VOLTAS,1530.0,1.63,8.22,10.60,19.69,21996.0,15768.0,207510.0,-2.32,53.0,X-MC,2.07,78.0,0.72,1.45,15.58,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-9.50,-10.62,23.94,10.78,51425.0,-25510.0,214808.0,-46.87,18.0,X-LC,2.46,5.0,-0.50,1.50,0.00,X40,NTT,FMCG
52,PAGEIND,51605.08,-1.10,-9.77,44.75,30.62,95720.0,-23154.0,213900.0,-35.86,34.0,X-MC,15.33,55.0,-0.24,1.49,0.24,X40,ATH,APPARELS
3,ACC,3906.00,0.16,-26.85,124.42,64.15,216553.0,-63901.0,174050.0,-57.19,37.0,X-SC,5.80,82.0,-0.30,1.21,0.89,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.53,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,39.0,X-SC,17.32,92.0,-0.52,0.55,1.06,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,-0.63,-47.96,192.65,52.31,145543.0,-69612.0,75548.0,-44.63,43.0,X-SC,7.43,91.0,-0.48,0.53,1.12,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,1.38,-29.78,103.43,42.84,47212.0,-19360.0,45646.0,-54.27,45.0,X-SC,31.13,83.0,-0.41,0.32,3.27,XY24,NTT,MISC
58,RELAXO,1176.00,-0.63,-47.96,192.65,52.31,145543.0,-69612.0,75548.0,-44.63,43.0,X-SC,7.43,91.0,-0.48,0.53,1.12,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.53,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,39.0,X-SC,17.32,92.0,-0.52,0.55,1.06,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.40,-0.54,24.75,24.08,31251.0,-683.0,126267.0,-8.54,51.0,X-SC,7.63,89.0,-0.02,0.88,18.97,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.90,-18.71,76.31,43.32,101034.0,-30472.0,132400.0,-30.16,34.0,X-SC,7.49,90.0,-0.30,0.92,1.66,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,0.54,-11.36,36.18,20.71,121295.0,-42983.0,335254.0,-23.62,50.0,X-LC,2.91,1.0,-0.35,2.34,12.02,X40,ATH,IT
41,INFY,1972.00,0.76,10.80,21.16,34.24,73357.0,33791.0,346679.0,-13.85,55.0,X-LC,7.41,2.0,0.46,2.42,19.98,X40,NTT,IT
75,TMPV,600.00,0.07,-24.66,63.22,22.98,107600.0,-55699.0,170199.0,-21.86,59.0,X-LC,5.68,3.0,-0.52,1.19,7.05,XY24,NTT,AUTO
81,VBL,671.64,1.11,-1.05,37.11,35.67,115978.0,-3318.0,312524.0,-12.80,62.0,X-LC,2.91,4.0,-0.03,2.18,12.66,X40N,ATH,FMCG
43,ITC,452.00,-9.50,-10.62,23.94,10.78,51425.0,-25510.0,214808.0,-46.87,18.0,X-LC,2.46,5.0,-0.50,1.50,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.99,-34.29,102.25,32.89,51871.0,-26476.0,50730.0,-726.63,68.0,L-MC,6.72,259.0,-0.51,0.35,39.71,XR,NTT,BANKS
49,MASFIN,398.61,-1.27,-2.17,24.76,22.05,23732.0,-2130.0,95850.0,-15.54,53.0,H-SC,6.25,164.0,-0.09,0.67,39.32,XR,ATH,FINANCE
13,BSOFT,831.70,-0.44,-21.79,92.79,50.79,102075.0,-30646.0,110007.0,0.50,54.0,H-SC,10.81,151.0,-0.30,0.77,28.08,XR,ATH,IT
69,SURYODAY,216.00,0.99,-18.97,51.85,23.03,75232.0,-33976.0,145095.0,57.89,55.0,H-SC,6.58,167.0,-0.45,1.01,43.96,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,3.57,-15.85,18.86,0.02,27703.0,-27669.0,146890.0,123.77,50.0,M-SC,7.05,234.0,-1.00,1.02,27.77,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.0,14.42,-27.27,55.24,12.90,87783.0,-59584.0,158912.0,134.38,74.0,H-SC,11.73,161.0,-0.68,1.11,32.13,OX40N,NTT,FINANCE
13,BSOFT,831.7,-0.44,-21.79,92.79,50.79,102075.0,-30646.0,110007.0,0.50,54.0,H-SC,10.81,151.0,-0.30,0.77,28.08,XR,ATH,IT
36,ICICIPRULI,790.0,0.99,11.68,17.05,30.73,37513.0,23012.0,220017.0,-14.38,72.0,X-MC,4.79,75.0,0.61,1.53,25.94,X40,ATH,INSURANCE
74,TITAGARH,1548.0,-1.11,-16.96,75.50,45.74,151836.0,-41070.0,201107.0,10.54,61.0,H-SC,4.43,174.0,-0.27,1.40,30.80,XY24,NTT,ENGINEERING
84,WIPRO,420.0,1.47,9.57,57.22,72.27,106240.0,16221.0,185669.0,-6.08,62.0,M-LC,5.43,99.0,0.15,1.29,16.34,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.04
1,25,44.06
2,50,76.44


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.89
MC    30.78
LC    26.35
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.75
X40      23.89
X40N     12.82
XY25     10.49
XR        8.97
AR        8.57
OX40N     7.56
SR        0.97
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.32
X-LC    22.95
H-SC    22.83
M-SC    11.44
X-SC     7.81
H-MC     4.70
M-MC     1.41
M-LC     1.29
H-LC     1.08
L-LC     1.03
L-SC     0.81
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.06,-7.63,42.95
IT,12.77,-16.79,75.70
FINANCE,11.08,-12.14,59.21
MISC,6.96,-30.06,83.26
ELECTRICAL,5.80,-12.75,54.05
PAINTS,5.72,-12.28,28.96
INSURANCE,4.72,0.05,34.15
PHARMA,3.97,-3.13,35.60
AUTO,2.88,-27.16,71.24


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3133360.0,21
AR,1342697.0,10
XR,1203642.0,12
X40,1093188.0,15
X40N,901983.0,9
OX40N,706628.0,10
XY25,387587.0,7
SR,280550.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3518004.0,24
X-MC,1492106.0,16
M-SC,1417037.0,15
X-LC,932085.0,13
X-SC,799403.0,8
H-MC,405246.0,3
L-SC,174487.0,2
M-LC,106240.0,1
H-LC,79003.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1225005.0      6
           AR         920550.0      5
M-SC       XY24       802972.0      6
H-SC       XR         781542.0      7
X-MC       X40        501420.0      7
           XY24       433732.0      3
X-LC       X40        405074.0      6
X-MC       X40N       371752.0      4
X-SC       X40N       317693.0      3
H-SC       OX40N      310357.0      4
M-SC       OX40N      306551.0      5
X-SC       XY24       295016.0      3
H-SC       SR         280550.0      2
H-MC       AR         214852.0      2
X-LC       X40N       212538.0      2
H-MC       XY24       190394.0      1
X-SC       X40        186694.0      2
X-LC       XY24       186241.0      2
X-MC       XY25       185202.0      2
M-SC       XR         179222.0      2
           AR         128292.0      2
X-LC       XY25       128232.0      3
M-LC       XR         106240.0      1
L-SC       OX40N       89720.0      1
           XR          84767.0      1
H-LC       AR          79003.0      1
L-MC       XR          51871.0      1
M-MC       XY25        42909.0      1
L-LC       XY25        31244.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
